In [1]:
import pathlib
import numpy as np
import skimage
import plotly.graph_objects as go

from scipy.spatial import ConvexHull

import matplotlib.pyplot as plt
from matplotlib.path import Path as pltpath
from matplotlib.gridspec import GridSpec
from matplotlib.colors import ListedColormap
import matplotlib.colors as mcolors
import matplotlib.cm as mcm


import open3d as o3d
from copy import deepcopy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
from pin_regions import RgbdPinFetcher, SfMPinFetcher

In [5]:
dataset_root = pathlib.Path(r'/home/crest/w/hwang_Pro/datasets/3DPotatoTwin')
pin_ref_folder = pathlib.Path(r'/home/crest/Documents/Github/PotatoScan/3dscan/03_sfm_rgbd_registration/pin_ref')

In [6]:
rgbd_fetcher = RgbdPinFetcher(dataset_root)
sfm_fetcher = SfMPinFetcher(dataset_root, pin_ref_folder)

loading annotations into memory...
Done (t=0.22s)
creating index...
index created!


In [7]:
potato_id = 'R3-3'

rgbd_pcd, rgbd_pin_pcd = rgbd_fetcher.get(potato_id, visualize=True)
sfm_pcd, sfm_pin_pcd, sfm_pin_idx = sfm_fetcher.get(potato_id, visualize=True)

Thresh=0.35 get pin convex hull volumn 96018.77215018909 > 60, denoise first
Thresh=0.3 get pin convex hull volumn 92337.8503708364 > 60, denoise first
Thresh=0.25 get pin convex hull volumn 66611.8752524912 > 60, denoise first
Thresh=0.2 get pin convex hull volumn 3900.371686427296 > 60, denoise first
Stop at thresh=0.2 with hull volume = 57.30333822858874 after denoising
